In [2]:
%load_ext autoreload
%autoreload 2

import warnings
warnings.filterwarnings('ignore')

In [3]:
import typing
import io
import os

import torch
import numpy as np
import cv2
import matplotlib.pyplot as plt

from urllib.request import urlretrieve

from PIL import Image
#from torchvision import transforms
from models.transformers import VisionTransformer, CONFIGS

True


In [3]:
model_type = 'ViT-L_32'
config = CONFIGS[model_type]
model = VisionTransformer(config, img_size=32, num_classes=2,vis = True)
model.load_state_dict(torch.load('/data1/fog/TransMAP/EXPs/EX004_2D_FiveTop_Patch/best_model_EX004_2D_FiveTop_Patch.pth'))
model.eval()

VisionTransformer(
  (transformer): Transformer(
    (embeddings): Embeddings(
      (patch_embeddings): Conv2d(5, 320, kernel_size=(8, 8), stride=(8, 8))
      (dropout): Dropout(p=0.3, inplace=False)
    )
    (encoder): Encoder(
      (layer): ModuleList(
        (0): Block(
          (attention_norm): LayerNorm((320,), eps=1e-06, elementwise_affine=True)
          (ffn_norm): LayerNorm((320,), eps=1e-06, elementwise_affine=True)
          (ffn): Mlp(
            (fc1): Linear(in_features=320, out_features=512, bias=True)
            (fc2): Linear(in_features=512, out_features=320, bias=True)
            (dropout): Dropout(p=0.3, inplace=False)
          )
          (attn): Attention(
            (query): Linear(in_features=320, out_features=320, bias=True)
            (key): Linear(in_features=320, out_features=320, bias=True)
            (value): Linear(in_features=320, out_features=320, bias=True)
            (out): Linear(in_features=320, out_features=320, bias=True)
           

In [4]:
from src import dataloader
import json

DEFAULT_IMAGE_DIR_NAME    = '/data1/fog-data/fog-maps/'
DEFAULT_TARGET_DIR_NAME   = '/data1/fog/Dataset/TARGET'
year_information          = {'2009':['20090101', '20091231'],
                            '2010':['20100101', '20101231'],
                            '2011':['20110101', '20111231'],
                            '2012':['20120101', '20121231'],
                            '2013':['20130101', '20131231'],
                            '2014':['20140101', '20141231'],
                            '2015':['20150101', '20151231'],
                            '2016':['20160101', '20161231'],
                            '2017':['20170101', '20171231'],
                            '2018':['20180101', '20181231'],
                            '2019':['20190101', '20191231'],
                            '2020':['20200101', '20201215']}

#data_split_dict = 
            
dataset = dataloader.input_dataframe_generater(img_path = None, 
                                            target_path = None, 
                                            first_date_string = year_information['2009'][0], 
                                            last_date_string  = year_information['2020'][1], 
                                            target_binarizing_thre = 1).dataframe_generation()

# split the data into train, validation and test:
train_df, valid_df, test_df = dataloader.split_data_train_valid_test(dataset, year_split_dict = {'train': ['2013', '2014', '2015', '2016', '2017'], 'valid': ['2009', '2010', '2011'], 'test': ['2018', '2019', '2020']})
with open('/data1/fog/TransMAP/EXPs/EX002_2D_FiveTop/mean_std_EX002_2D_FiveTop.json', 'r') as file:
    norm_mean_std_dict = json.load(file)


test_dataset = dataloader.DataAdopter(test_df, 
                                        map_structure         = '2D', 
                                        predictor_names       = dataloader.NETCDF_PREDICTOR_NAMES['Five_Top'], 
                                        lead_time_pred        = 24, 
                                        mean_std_dict         = norm_mean_std_dict,
                                        point_geolocation_dic = None)

In [26]:
x = test_dataset[1977]['input']

In [27]:
x.shape

torch.Size([5, 32, 32])

In [16]:
logits, att_mat = model(x.unsqueeze(0))
att_mat = torch.stack(att_mat).squeeze(1)
# Average the attention weights across all heads.
att_mat = torch.mean(att_mat, dim=1)
# To account for residual connections, we add an identity matrix to the
# attention matrix and re-normalize the weights.
residual_att = torch.eye(att_mat.size(1))
aug_att_mat = att_mat + residual_att
aug_att_mat = aug_att_mat / aug_att_mat.sum(dim=-1).unsqueeze(-1)

# Recursively multiply the weight matrices
joint_attentions = torch.zeros(aug_att_mat.size())
joint_attentions[0] = aug_att_mat[0]

for n in range(1, aug_att_mat.size(0)):
    joint_attentions[n] = torch.matmul(aug_att_mat[n], joint_attentions[n-1])
    
# Attention from the output token to the input space.
v = joint_attentions[-1]
grid_size = int(np.sqrt(aug_att_mat.size(-1)))
mask = v[0, 1:].reshape(grid_size, grid_size).detach().numpy()

print(mask.shape)

(4, 4)


In [42]:
mask = cv2.resize((mask / mask.max()), (32, 32))[..., np.newaxis]
result = (mask * x.detach().numpy()[0, :, :]).astype("uint8")

In [46]:
result.shape

(32, 32, 32)